In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tsunamibayes as tb
from clawpack.geoclaw import dtopotools
import pandas as pd
import os.path

In [3]:
bounds = {'lat_min':-10,'lat_max':-2,'lon_min':126,'lon_max':133.5}
# depth_path = 'sum_slab2_dep_02.23.18.xyz'
# dip_path = 'sum_slab2_dip_02.23.18.xyz'
# strike_path = 'sum_slab2_str_02.23.18.xyz'
# # arrays = tb.fault.load_slab2_data(depth_path,dip_path,strike_path,bounds)
arrays = np.load('/Users/nephi/tsunamibayes/scenarios/banda_1852/data/slab2_banda.npz')
banda = tb.GridFault(bounds=bounds,**arrays)


In [4]:
# Get sample quakes from previous runs and load the first quake
quakes = pd.read_csv("/Users/nephi/Downloads/model_params.csv")
q = quakes.values[40]
print(quakes.columns)


#Set variables for subfault split
qlat = q[1]
qlon = q[2]
qlen = q[3]
qwidth = q[4]
qslip = q[5]
qstrike = q[6]
qdip = q[7]
qdep = q[8]
qrake = q[9]

print(q)

Index(['Unnamed: 0', 'latitude', 'longitude', 'length', 'width', 'slip',
       'strike', 'dip', 'depth', 'rake', 'depth_offset'],
      dtype='object')
[ 4.00000000e+01 -5.90332508e+00  1.31482588e+02  1.78841331e+05
  1.05840013e+05  9.87355975e+00  2.04277484e+02  2.41976461e+01
  3.10853093e+04  9.00000000e+01  1.28804197e+00]


In [5]:
# np.savez('slab2_banda.npz',**arrays)

#get the subfault split
subfaults = banda.subfault_split(qlat,qlon,qlen,qwidth,qslip)
print(subfaults)

print(type(subfaults))
#Clear out each row we don't need and fill those which are necessary
subfaults['type'] = 0.5
subfaults['risetime'] = 0.5
subfaults['depth'] = subfaults['depth']/100
header = ['longitude', 'latitude','depth','strike','dip','type','risetime','length','width']

#Set precision for each data entry
subfaults['depth'] = subfaults['depth'].apply(lambda x: round(x,3))
subfaults['strike'] = subfaults['strike'].apply(lambda x: round(x,2))
subfaults['dip'] = subfaults['dip'].apply(lambda x: round(x,2))
subfaults['length'] = subfaults['length'].apply(lambda x: round(x,2))
subfaults['width'] = subfaults['width'].apply(lambda x: round(x,2))


#Write our .fault file
subfaults.to_string("/Users/nephi/Desktop/FakeQuakes_tutorial/files/banda.fault",columns = header, header=False)

#Print to check
#of = open("banda.fault","r+")
#print(of.read())

subfaults.to_csv('subfaultspd.csv')


    latitude   longitude      strike        dip         depth        length  \
0  -5.160630  131.917892  172.666911  16.156845  20431.511201  16258.302802   
1  -5.345487  131.925958  181.031495  16.021851  20518.541856  16258.302802   
2  -5.527939  131.910219  189.338614  15.925217  20452.922616  16258.302802   
3  -5.703677  131.873323  197.112530  15.948927  20250.931887  16258.302802   
4  -5.869708  131.819576  203.669325  16.099670  19898.871946  16258.302802   
5  -6.025043  131.753936  208.561856  16.331247  19454.097052  16258.302802   
6  -6.170977  131.681816  211.768832  16.552027  18962.572918  16258.302802   
7  -6.308916  131.605123  213.596102  16.730282  18478.962394  16258.302802   
8  -6.441122  131.525886  214.442821  16.863737  18063.001137  16258.302802   
9  -6.569705  131.444984  214.730889  16.930959  17692.738337  16258.302802   
10 -6.696438  131.362366  214.861006  16.948909  17393.205629  16258.302802   
11 -5.195067  131.619411  173.393574  21.528413  312

In [31]:
!pwd

/Users/nephi/tsunamibayes/testing_notebooks


In [6]:
Lat,Lon = np.meshgrid(banda.lat,banda.lon,indexing='ij')

In [7]:
lon = 131.46686782132107
lat = -3.994735053198803
mag = 8.828132593154196
delta_logl = -0.018864928329097086
delta_logw = -0.08955551930345311
depth_offset = 10.764981349160156

In [8]:
length = tb.utils.calc_length(mag, delta_logl)
width = tb.utils.calc_width(mag, delta_logw)
slip = tb.utils.calc_slip(mag, length, width)

In [9]:
subfault_params = banda.subfault_split2(lat,lon,length,width,slip,depth_offset=0,slip_dist='elliptical',rake='parallel',n=5)

In [10]:
subfault_params

,latitude,longitude,strike,dip,depth,slip,length,width,rake
0,-2.710870,129.678279,105.001110,4.081236,5422.493203,1.139577,45477.690977,28960.47848,parallel
1,-2.965831,129.631994,102.761752,6.016862,7917.993400,2.649596,45477.690977,28960.47848,parallel
2,-3.219335,129.585964,100.277028,9.810106,11757.156085,3.510145,45477.690977,28960.47848,parallel
3,-3.468712,129.540669,98.615270,17.722100,18423.423240,2.649596,45477.690977,28960.47848,parallel
4,-3.704565,129.497821,98.864036,29.281073,29890.071015,1.139577,45477.690977,28960.47848,parallel
5,-2.814285,130.124216,106.862858,4.134728,5299.261531,2.041852,45477.690977,28960.47848,parallel
6,-3.063706,130.054108,106.458442,6.232386,7854.021346,4.747449,45477.690977,28960.47848,parallel
7,-3.311501,129.984441,105.680317,10.361717,11873.718645,6.289348,45477.690977,28960.47848,parallel
8,-3.554873,129.915999,105.381599,18.295173,18835.212736,4.747449,45477.690977,28960.47848,parallel
9,-3.785386,129.851156,106.352537,28.766214,30383.689062,2.041852,45477.690977,28960.47848,parallel


In [11]:
fault = tb.maketopo.make_fault_dtopo(subfault_params,bounds,verbose=True)

[<clawpack.geoclaw.dtopotools.SubFault object at 0x15d89d890>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d89da90>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d889a90>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d89fa50>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d89dcd0>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d89fc50>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d89fc90>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d89d4d0>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d89fb10>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d89fb50>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d89de10>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d89fdd0>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d89fcd0>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d89fd10>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d89fed0>, <clawpack.geoclaw.dtopotools.SubFault object at 0x15d89fd90>, <clawpa

TypeError: can't multiply sequence by non-int of type 'float'

## Uniform slip, rake = 90

In [ ]:
dtopo = fault.dtopo
X,Y = np.meshgrid(dtopo.x,dtopo.y)
fault.plot_subfaults(slip_color=True,plot_rake=True)
# plt.pcolormesh(X,Y,dtopo.dZ[0])
# plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
# plt.clabel(cs)
plt.show()

In [ ]:
# fault.plot_subfaults(slip_color=True)
plt.pcolormesh(X,Y,dtopo.dZ[0])
plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
plt.show()

## Uniform slip, parallel slip vectors (variable rake)

In [ ]:
dtopo = fault.dtopo
X,Y = np.meshgrid(dtopo.x,dtopo.y)
fault.plot_subfaults(slip_color=True,plot_rake=True)
# plt.pcolormesh(X,Y,dtopo.dZ[0])
# plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
# plt.clabel(cs)
plt.show()

In [ ]:
# fault.plot_subfaults(slip_color=True)
plt.pcolormesh(X,Y,dtopo.dZ[0])
plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
# plt.clabel(cs)
plt.show()

## Elliptical slip, rake=90

In [ ]:
dtopo = fault.dtopo
X,Y = np.meshgrid(dtopo.x,dtopo.y)
fault.plot_subfaults(slip_color=True,plot_rake=True)
# plt.pcolormesh(X,Y,dtopo.dZ[0])
# plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
# plt.clabel(cs)
plt.show()

In [ ]:
# fault.plot_subfaults(slip_color=True)
plt.pcolormesh(X,Y,dtopo.dZ[0])
plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
# plt.clabel(cs)
plt.show()

## Elliptical slip, variable rake

In [ ]:
dtopo = fault.dtopo
X,Y = np.meshgrid(dtopo.x,dtopo.y)
fault.plot_subfaults(slip_color=True,plot_rake=True)
# plt.pcolormesh(X,Y,dtopo.dZ[0])
# plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
# plt.clabel(cs)
plt.show()

In [ ]:
# fault.plot_subfaults(slip_color=True)
plt.pcolormesh(X,Y,dtopo.dZ[0])
plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
# plt.clabel(cs)
plt.show()

In [ ]:
dtopo.dZ.shape

In [ ]:
fault.plot_subfaults()

In [ ]:
fault.plot_subfaults_depth()

In [ ]:
fault?

In [ ]:
fault.plot_subfaults?

In [ ]:
def get_length(deltalogl, mag):
    """ Length is sampled from a truncated normal distribution that
    is centered at the linear regression of log10(length_cm) and magnitude.
    Linear regression was calculated from wellscoppersmith data.

    Parameters:
    mag (float): the magnitude of the earthquake

    Returns:
    length (float): Length in meters. a sample from the normal distribution centered on the regression
    """

    m = 0.5233956445903871       # slope
    c = 1.0974498706605313     # y intercept

    mu_logl = m*mag + c
    logl = mu_logl + deltalogl
    return 10**logl

def get_width(deltalogw, mag):
    """
    Parameters:
    mag (float): the magnitude of the earthquake

    Returns:
    width (float): width in meters
    """
    m = 0.29922483873212863       # slope
    c = 2.608734705074858     # y intercept

    mu_logw = m*mag + c
    logw = mu_logw + deltalogw
    return 10**logw

In [ ]:
def get_slip(length, width, mag):
    """Calculated from magnitude and rupture area, Ron Harris gave us the equation
        Parameters:
        Length (float): m
        Width (float): m
        mag (float): moment magnitude

        Return:
        slip (float): meters
        """
    #Dr. Harris' rigidity constant : 3.2e11 dynes/cm^2
    mu_dyn_cm2 = 3.e11
    mu = mu_dyn_cm2 * 1e-5 * 1e4 #convert to N/m^2
    slip = 10**(3/2 * ( mag + 6.06 )) / (mu * length * width)
    return slip

In [ ]:
mag = 8.786524
lon = 132.048556
lat = -4.558317
deltalogl = -0.011334
deltalogw = 0.017210
deltadepth = 1020.248

In [ ]:
length = get_length(deltalogl,mag)
width = get_width(deltalogw,mag)
slip = get_slip(length,width,mag)
print(length,width,slip)

In [ ]:
subfault_params = banda.subfault_split(lat,lon,length,width,slip,depth_offset=deltadepth)

In [ ]:
fault = tb.maketopo.make_fault_dtopo(subfault_params,bounds)

In [ ]:
fault.plot_subfaults()
cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
plt.clabel(cs)
plt.show()

In [ ]:
d = banda.depth.copy()
d[d==0] = np.nan

In [ ]:
Lat,Lon = np.meshgrid(banda.lat,banda.lon)
plt.pcolormesh(Lon,Lat,d.T)
plt.scatter([132.225436],[-3.881648])

In [ ]:
banda.depth_map(-3.881648,132.225436)

In [ ]:
subfault_params.loc[4]

In [ ]:
def split_rect(fault,lat,lon,length,width,deltadepth,n=11,m=3):
    R = fault.R
    # n = int(length/15000)
    # m = int(width/15000)
    n_steps = 8
    length_step = length/(n*n_steps)
    width_step = width/(m*n_steps)
    sublength = length/n
    subwidth = width/m

    lats = np.empty(n)
    lons = np.empty(n)
    lats[(n - 1)//2] = lat
    lons[(n - 1)//2] = lon

    # add strikeward and anti-strikeward centers
    bearing1 = fault.strike_from_lat_lon(lat,lon)
    bearing2 = (bearing1-180)%360
    lat1,lon1 = lat,lon
    lat2,lon2 = lat,lon
    for i in range(1,(n - 1)//2+1):
        for j in range(n_steps):
            lat1,lon1 = Fault.displace(lat1,lon1,bearing1,length_step,R)
            lat2,lon2 = Fault.displace(lat2,lon2,bearing2,length_step,R)
            bearing1 = fault.strike_from_lat_lon(lat1, lon1)
            bearing2 = (fault.strike_from_lat_lon(lat2, lon2)-180)%360
        lats[(n-1)//2+i] = lat1
        lats[(n-1)//2-i] = lat2
        lons[(n-1)//2+i] = lon1
        lons[(n-1)//2-i] = lon2

    strikes = fault.strike_map(np.vstack((lats,lons)).T)
    dips = fault.dip_map(np.vstack((lats,lons)).T)
    dipward = (strikes+90)%360

    Lats = np.empty((m,n))
    Lons = np.empty((m,n))
    Strikes = np.empty((m,n))
    Dips = np.empty((m,n))
    Lats[(m-1)//2] = lats
    Lons[(m-1)//2] = lons
    Strikes[(m-1)//2] = strikes
    Dips[(m-1)//2] = dips

    # add dipward and antidipward centers
    templats1,templons1 = lats.copy(),lons.copy()
    templats2,templons2 = lats.copy(),lons.copy()
    tempdips1,tempdips2 = dips.copy(),dips.copy()
    for i in range(1,(m - 1)//2+1):
        for j in range(n_steps):
            templats1,templons1 = Fault.displace(templats1,templons1,dipward,width_step*np.cos(np.deg2rad(tempdips1)),R)
            templats2,templons2 = Fault.displace(templats2,templons2,dipward,-width_step*np.cos(np.deg2rad(tempdips2)),R)
            tempdips1 = fault.dip_map(np.vstack((templats1,templons1)).T)
            tempdips2 = fault.dip_map(np.vstack((templats2,templons2)).T)
        Lats[(m-1)//2+i] = templats1
        Lats[(m-1)//2-i] = templats2
        Lons[(m-1)//2+i] = templons1
        Lons[(m-1)//2-i] = templons2
        Strikes[(m-1)//2+i] = fault.strike_map(np.vstack((templats1,templons1)).T)
        Strikes[(m-1)//2-i] = fault.strike_map(np.vstack((templats2,templons2)).T)
        Dips[(m-1)//2+i] = tempdips1
        Dips[(m-1)//2-i] = tempdips2

    Depths = fault.depth_map(np.vstack((Lats.flatten(),Lons.flatten())).T) + deltadepth
    data = [Lats,Lons,Strikes,Dips,Depths]
    data = [arr.flatten() for arr in data]
    return np.array(data).T, sublength, subwidth